# Group Project: Heart Attach Analysis & Prediction
### By: Halee Staggs, Vicky van der Wagt, Jesse Gutierrez

Code URL: https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?resource=download 

#### Dataset Summary Breakdown

**Age** : Age of the patient

**Sex** : Sex of the patient

**exang**: exercise induced angina (1 = yes; 0 = no)

**ca**: number of major vessels (0-3)

**cp**: Chest Pain type chest pain type

**Value 1**: typical angina

**Value 2**: atypical angina

**Value 3**: non-anginal pain

**Value 4**: asymptomatic

**trtbps**: resting blood pressure (in mm Hg)

**chol**: cholestoral in mg/dl fetched via BMI sensor

**fbs**: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

**rest_ecg**: resting electrocardiographic results

**Value 0**: normal

**Value 1**: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)

**Value 2**: showing probable or definite left ventricular hypertrophy by Estes' criteria

**thalach**: maximum heart rate achieved

**target**: 0= less chance of heart attack 1= more chance of heart attack

In [1]:
# As the followed code did not work, I had to install the following package
# %load_ext rpy2.ipython
# I installed this in the terminal: conda install -c r r-essentials, conda install -c r rpy2, pip install tzlocal & pip install --upgrade rpy2, pip install --force-reinstall cffi

In [9]:
# Had to load the following command to generate R in my vs code jupyter notebook
%load_ext rpy2.ipython


In [13]:
%%R

data <- read.csv('/Users/jesse/Desktop/School/University of San Diego/ADS 502- Applied Data Mining/Group Project/heart.csv')
head(data)

  age sex cp trtbps chol fbs restecg thalachh exng oldpeak slp caa thall output
1  63   1  3    145  233   1       0      150    0     2.3   0   0     1      1
2  37   1  2    130  250   0       1      187    0     3.5   0   0     2      1
3  41   0  1    130  204   0       0      172    0     1.4   2   0     2      1
4  56   1  1    120  236   0       1      178    0     0.8   2   0     2      1
5  57   0  0    120  354   0       1      163    1     0.6   2   0     2      1
6  57   1  0    140  192   0       1      148    0     0.4   1   0     1      1


In [18]:
%%R
# Sought to identify the total observations & attributes of the DS
dim(data)

[1] 303  14


In [21]:
%%R
# Wanted to see if there was any null values
sum(is.na(data))

[1] 0
